In [ ]:
import cv2
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
import matplotlib.pyplot as plt
from easydict import EasyDict as edict
import parser
import os
import network
import warnings
import torch.nn.functional as F
warnings.filterwarnings('ignore')
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
t = transforms.Compose([transforms.Resize((224, 224)), 
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
args = edict({'brightness': None,
 'cache_refresh_rate': 1000,
 'contrast': None,
 'criterion': 'triplet',
 'dataset_name': 'st_lucia',
 'datasets_folder': './datasets',
 'dense_feature_map_size': [61, 61, 128],
 'device': 'cuda',
 'efficient_ram_testing': False,
 'epochs_num': 50,
 'features_dim': 1024,
 'foundation_model_path': None,
 'horizontal_flip': False,
 'hue': None,
 'infer_batch_size': 16,
 'l2': 'before_pool',
 'lr': 1e-05,
 'majority_weight': 0.01,
 'margin': 0.1,
 'mining': 'partial',
 'neg_samples_num': 1000,
 'negs_num_per_query': 2,
 'num_workers': 8,
 'optim': 'adam',
 'patience': 3,
 'pca_dataset_folder': None,
 'pca_dim': None,
 'queries_per_epoch': 5000,
 'rand_perspective': None,
 'random_resized_crop': None,
 'random_rotation': None,
 'recall_values': [1, 5, 10, 20],
 'registers': False,
 'rerank_num': 100,
 'resize': [224, 224],
 'resume': 'ckpts/SelaVPR_msls.pth',
 'saturation': None,
 'save_dir': 'test/default/2025-02-06_13-22-01',
 'seed': 0,
 'test_method': 'hard_resize',
 'train_batch_size': 4,
 'train_positives_dist_threshold': 10,
 'val_positive_dist_threshold': 25})
model = network.GeoLocalizationNet(args)
model = model.to(args.device)
model = torch.nn.DataParallel(model)
state_dict = torch.load(args.resume)["model_state_dict"]
model.load_state_dict(state_dict)

In [ ]:
imgpath = "/robodata/smodak/VPR/SelaVPR/datasets/san_francisco/images/test/database/@0550313.87@4184192.09@10@S@037.80373@-122.42845@14666@00@089@003@@@@@.jpg"
img = Image.open(imgpath).convert('RGB')
H, W = np.asarray(img).shape[:2]  # Original image dimensions
img = t(img).unsqueeze(0).to(args.device)
with torch.inference_mode():
    fm = model.module.backbone(img)["x_norm_patchtokens"]
fm2 = fm.squeeze().reshape(16, 16, 1024)
A = fm2.mean(dim=-1)
A_resized = F.interpolate(A.unsqueeze(0).unsqueeze(0), size=(H, W), mode='bilinear', align_corners=False)
A_resized = A_resized.squeeze().cpu().detach().numpy()
A_resized = np.abs(A_resized)

In [ ]:
def visualize_attention_map(image, attention_map):
    fig, ax = plt.subplots()
    ax.imshow(image)  # Display original image
    im = ax.imshow(attention_map, cmap='jet', alpha=0.5)  # Overlay heatmap with transparency
    ax.axis('off')
    # Add colorbar
    cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.ax.tick_params(labelsize=8)  # Adjust colorbar label size for readability
    plt.show()
visualize_attention_map(Image.open(imgpath), A_resized)

In [1]:
from match_image import *
import os

xFormers not available
xFormers not available


In [2]:
sela = SelaVPRminimal()
img_dir = "bags/route"
img_list = sorted(os.listdir(img_dir))
img_paths = [os.path.join(img_dir, img) for img in img_list]

In [18]:
query_index = 18
candidate_indices = [3, 4, 5, 6, 7, 10, 20, 100]

In [19]:
query_imgpath = img_paths[query_index]
candidate_imgpaths = [img_paths[i] for i in candidate_indices]

In [20]:
indices, scores = sela.match_image(query_imgpath, candidate_imgpaths, retscores=True)
print([candidate_indices[i] for i in indices.squeeze().tolist()])

reranking...


100%|██████████| 1/1 [00:00<00:00,  1.57it/s]

[20, 5, 3, 7, 4, 6, 10, 100]


In [21]:
indices.squeeze().tolist()

[6, 2, 0, 4, 1, 3, 5, 7]

In [22]:
scores

[1386.0]

In [ ]:
import numpy as np

arr = np.array([30, 10, 50, 40])
sorted_indices = arr.argsort()[::-1]

print(sorted_indices)  # Indices of elements sorted in descending order
print(arr[sorted_indices])  # Elements sorted in descending order


In [ ]:
arr.argsort()[::-1]

In [ ]:
import numpy as np

# Example usage
list1 = [100, 250, 400]
list2 = [90, 200, 350, 500]

matched_list = match_closest(list1, list2)
print(matched_list)  # Closest values from list2


In [ ]:
# Example usage
list1 = [100, 200, 300]
list2 = [110, 150, 180, 210, 250, 290]

grouped = group_by_range(list1, list2)
print(grouped)


In [ ]:
import numpy as np
import os
rootdir = "/root/SelaVPR/exps/room"
all_imgs = sorted(os.listdir(os.path.join(rootdir, "images")))
all_imgs_t = [int(a.split(".")[0]) for a in all_imgs]

In [ ]:
def read_cmd_vel_file(file_path):
    data_dict = {}
    with open(file_path, 'r') as f:
        next(f)  # Skip header
        for line in f:
            values = line.split()
            timestamp = int(values[0])
            data_dict[timestamp] = list(map(float, values[1:]))  # Convert remaining values to float
    return data_dict
all_cmdvel = read_cmd_vel_file(os.path.join(rootdir, "cmd_vel.txt"))
all_cmdvel_t = list(all_cmdvel.keys())

In [ ]:
def read_localization_file(file_path):
    data_dict = {}
    with open(file_path, 'r') as f:
        next(f)  # Skip header
        for line in f:
            values = line.split()
            timestamp = int(values[0])
            data_dict[timestamp] = list(map(float, values[1:]))  # Convert x, y, theta to floats
    return data_dict
all_localization = read_localization_file(os.path.join(rootdir, "localization.txt"))
all_localization_t = list(all_localization.keys())

In [ ]:
def match_closest(list1, list2):
    list1 = np.array(list1)
    list2 = np.array(list2)
    closest_indices = np.abs(list1[:, None] - list2).argmin(axis=1)
    return list2[closest_indices]

def group_by_range(list1, list2):
    result = {}
    j = 0  # Pointer for list2

    for i in range(len(list1) - 1):
        start, end = list1[i], list1[i + 1]
        result[start] = []

        while j < len(list2) and list2[j] < end:
            if list2[j] >= start:
                result[start].append(list2[j])
            j += 1

    return result

In [ ]:
mm = group_by_range(all_imgs_t, all_cmdvel_t)

In [ ]:
all_cmdvel[mm[1739424888226575879][0]]

In [ ]:
import cv2
from PIL import Image
imgpath = "/root/SelaVPR/exps/room/images/1739424888226575879.png"
cv_image = cv2.imread(imgpath)
cv_image_rgb = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
pil_img = Image.fromarray(cv_image_rgb)
pil_img